In [4]:
from torch import nn, optim
import numpy as np
import pickle as p
import torch
from tqdm.notebook import trange
from tqdm import tqdm
from torchsummary import summary

from network import ConvNet
from sklearn.metrics import f1_score
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from dataset import CogDataset

In [5]:
# function for training and evaluation
def train(model, num_epochs, train_dataloader, val_dataloader):
    losses_train = []
    accuracies_train = []
    f1_train = []
    losses_val = []
    accuracies_val = []
    f1_val = []

    for epoch in trange(num_epochs, unit="epochs"):

        # training
        running_loss = 0.0
        running_accuracy = 0.0
        running_f1 = 0.0

        model.train()

        for input, target in train_dataloader:

            optimizer.zero_grad()

            output = model(input)
            
            loss = criterion(output, target)
            loss.backward()

            running_loss += loss.item()

            prediction = torch.argmax(output, dim=1)


            for p, t in zip(prediction, target):
                if p == t:
                    running_accuracy+=1 
            #running_f1 += f1_score(target, prediction, average=None)
        
        optimizer.step()

        running_loss /= len(train_dataloader)
        running_accuracy /= len(train_dataloader)
        #running_f1 /= len(train_dataloader)

        losses_train.append(running_loss)
        accuracies_train.append(running_accuracy)
        #f1_train.append(running_f1)

        # output
        tqdm.write('Epoch {} (train) -- loss: {:.4f} acc: {:.4f}'.format(epoch, running_loss, running_accuracy))


        # validation
        with torch.no_grad():

            model.eval()

            running_loss = 0.0
            running_accuracy = 0.0
            running_f1 = 0.0

            for input, target in val_dataloader:

                    output = model(input)
                    loss = criterion(output, target)

                    running_loss += loss.item()

                    prediction = torch.argmax(output, dim=1).float()   

                    for o, t in zip(prediction, target):
                        if o == t:
                            running_accuracy+=1 
                    #running_f1 += f1_score(target, prediction)

            running_loss /= len(val_dataloader)
            running_accuracy /= len(val_dataloader)
            #running_f1 /= len(val_dataloader)

            losses_val.append(running_loss)
            accuracies_val.append(running_accuracy)
            #f1_val.append(running_f1)

            # output
            tqdm.write('Epoch {} (valid) -- loss: {:.4f} acc: {:.4f}'.format(epoch, running_loss, running_accuracy))
    # make model callable outside this function
    train.trained_model = model
    train.losses_train = losses_train
    train.losses_val = losses_val

In [6]:
# load data and label files
# shape = [2284, 200, 3, 9] --> [datasets, time series, channels, devices]
data = p.load(open(r"data/train.pkl", "rb"))
labels = p.load(open(r"data/labels.pkl", "rb"))
print(data.shape)

# create datasets
train_dataset = CogDataset(data, labels, train=True)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=3, shuffle=True)

val_dataset = CogDataset(data, labels, train=False)
val_dataloader = DataLoader(dataset=val_dataset, batch_size=3, shuffle=False)


(2284, 8, 200, 3)


In [7]:
label_counts = torch.from_numpy(labels).bincount().float()
weights = np.sqrt(1/label_counts)
weights /= weights.mean()
print(weights)

tensor([0.9987, 0.9867, 0.9752, 0.9867, 0.9230, 0.9867, 0.9867, 0.9428, 0.9327,
        1.4299, 0.9532, 0.9987, 1.0111, 1.0111, 1.0111, 0.9867, 1.0111, 0.9987,
        0.9640, 1.0111, 0.9327, 0.9752, 1.0111, 0.9640, 0.9230, 1.4299, 0.9640,
        0.9987, 0.9640, 0.9987, 0.9640, 1.0240, 0.9987, 0.9752, 0.9752, 0.9428,
        1.0111, 1.0111, 0.9752, 0.9867, 0.9867, 0.9867, 0.9640, 1.0111, 1.0111,
        0.9752, 0.9532, 0.9987, 1.0111, 1.0240, 0.9987, 1.0111, 0.9640, 0.9987,
        0.9752])


In [10]:
# define loss, optimizer etc.
model = ConvNet()
criterion = nn.CrossEntropyLoss(weight = weights.double())
optimizer = optim.Adam(model.parameters())


In [11]:
# train model
train(model.double(), 20, train_dataloader, val_dataloader)

  0%|          | 0/20 [00:00<?, ?epochs/s]

Epoch 0 (train) -- loss: 4.0203 acc: 0.0722
Epoch 0 (valid) -- loss: 4.0193 acc: 0.0392
Epoch 1 (train) -- loss: 4.0148 acc: 0.0591
Epoch 1 (valid) -- loss: 4.0150 acc: 0.0654
Epoch 2 (train) -- loss: 4.0112 acc: 0.0591
Epoch 2 (valid) -- loss: 4.0058 acc: 0.0588
Epoch 3 (train) -- loss: 4.0065 acc: 0.0591
Epoch 3 (valid) -- loss: 3.9979 acc: 0.0588
Epoch 4 (train) -- loss: 3.9975 acc: 0.0722
Epoch 4 (valid) -- loss: 3.9865 acc: 0.0784
Epoch 5 (train) -- loss: 3.9918 acc: 0.0575
Epoch 5 (valid) -- loss: 3.9871 acc: 0.0654
Epoch 6 (train) -- loss: 3.9915 acc: 0.0739
Epoch 6 (valid) -- loss: 3.9787 acc: 0.0784
Epoch 7 (train) -- loss: 3.9903 acc: 0.0690
Epoch 7 (valid) -- loss: 3.9742 acc: 0.0915
Epoch 8 (train) -- loss: 3.9846 acc: 0.0739
Epoch 8 (valid) -- loss: 3.9805 acc: 0.0915
Epoch 9 (train) -- loss: 3.9823 acc: 0.0706
Epoch 9 (valid) -- loss: 3.9614 acc: 0.0915
Epoch 10 (train) -- loss: 3.9736 acc: 0.0821
Epoch 10 (valid) -- loss: 3.9642 acc: 0.0915
Epoch 11 (train) -- loss: 3.97